In [ ]:
import os
import pickle
import matplotlib.pyplot as plt

CKPT_DIR = "/content/drive/MyDrive/tomato/checkpoints"
file_path = os.path.join(CKPT_DIR, "hyperparam_results.pkl")

if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        results = pickle.load(f)

    # 绘图
    plt.figure(figsize=(12,5))
    for r in results:
        label = f"lr={r['lr']}, bs={r['batch_size']}, freeze_layer4={r['freeze_layer4']}"
        plt.plot(r['val_acc'], label=label)
    plt.title("Validation Accuracy Curves")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

    plt.figure(figsize=(12,5))
    for r in results:
        label = f"lr={r['lr']}, bs={r['batch_size']}, freeze_layer4={r['freeze_layer4']}"
        plt.plot(r['val_loss'], label=label)
    plt.title("Validation Loss Curves")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
else:
    print("结果文件不存在，请先运行训练脚本生成 hyperparam_results.pkl")
